## We need the package `PyCall` for importing Python packages and code

In [ ]:
# wrapping functions:
using PyCall #if calling this for the first time, add the package first by typing Pkg.add("PyCall")

In [ ]:
pushfirst!(PyVector(pyimport("sys")."path"), "") #to tell pyimport search in the local directory

### 1. take your existing python function and wrap it into a julia function
_filename: python_sum.py_

```
def py_sum(A):
    s = 0.0
    for a in A:
        s += a
    return s
```

In [ ]:
python_sum = pyimport("python_sum") # wrote a function out side julia

In [ ]:
m = [1,2,3]
python_sum.py_sum(m)

### 2. You can also write quick and easy python code from within julia

In [ ]:
py"""
def py_sum2(A):
    s = 0.0
    for a in A:
        s += a
    return s
"""
sum_py = py"py_sum2"

In [ ]:
sum_py([1,2,3])

## 3. You can also import your favorite python package

In [ ]:
pyimport("cvxpy")

```
# if the above doesn't work, and you get something like what I get when I add networkx, try:
using Conda
Conda.add("cvxpy")
```

In [ ]:
pyimport("networkx")

In [ ]:
my_cvxpy = pyimport("cvxpy")

In [ ]:
x1 = my_cvxpy.Variable(1, name="x1")

In [ ]:
obj = my_cvxpy.square(x1)

In [ ]:
#old syntax: @pyimport cvxpy.constraints as cvxpyconstraints
x1 = my_cvxpy.Variable(1, name="x1")

In [ ]:
cvxpyconstraints = pyimport("cvxpy.constraints")

Least squares example from the CVXPY page
https://www.cvxpy.org/examples/basic/least_squares.html

```
# Import packages.
import cvxpy as cp
import numpy as np

# Generate data.
m = 20
n = 15
np.random.seed(1)
A = np.random.randn(m, n)
b = np.random.randn(m)

# Define and solve the CVXPY problem.
x = cp.Variable(n)
cost = cp.sum_squares(A*x - b)
prob = cp.Problem(cp.Minimize(cost))
prob.solve()

# Print result.
print("\nThe optimal value is", prob.value)
print("The optimal x is")
print(x.value)
print("The norm of the residual is ", cp.norm(A*x - b, p=2).value)
```

In [ ]:
m = 20
n = 15
A = randn(m,n)
b = randn(m)
;

In [ ]:
x = my_cvxpy.Variable(n)
cost = my_cvxpy.sum_squares(A*x - b)
prob = my_cvxpy.Problem(my_cvxpy.Minimize(cost))
prob.solve()

In [ ]:
println("\nThe optimal value is", prob.value)
println("The optimal x is")
println(x.value)
println("The norm of the residual is ", my_cvxpy.norm(A*x - b, p=2).value)

# Wrapping C code with `ccall`

In [ ]:
?ccall

More references in this link:
https://docs.julialang.org/en/v1/manual/calling-c-and-fortran-code/index.html

In [ ]:
t = ccall((:clock, "libc"), Int32, ())

In [ ]:
ccall((:floor,"math"),Float64,(Float64,),10.1)

In [ ]:
ccall((:floor,"libm"),Float64,(Float64,),10.1)

## Now let's wrap our own `C` function

* Switch to editor `hello_world.c`

```
#include <stdio.h>
int hello_world()
{
    printf("Hello, World!\n");
    return 0;
}

int hello_world_repeated(int n)
{
    for (int i = 0; i<n; i++){
        printf("Hello, World!\n");
    }
    return 0;
}
```
* To compile:

```
$ gcc -g -O -c hello_world.c 
$ gcc -dynamiclib -o hello_world_lib.dylib hello_world.o
```

In [ ]:
ccall((:hello_world,"hello_world_lib.dylib"),
    Int64,
    (),
    )

In [ ]:
ccall((:hello_world_repeated,"hello_world_lib.dylib"),
    Int64,
    (Int64,),
    10)

## What if we have a `C++` function? Let's take a look

```
#include <stdint.h>

extern "C"
{
int64_t Fibonacci_I(int64_t n)
{
	int x = 0;
	int y = 1;
	int t;
	for(int i=1; i<n; ++i)
	{
	    t = x;
	    x = y;
	    y = t+y;
	}
	return y;
}
};
```

In [ ]:
if Sys.islinux()
    const libpath = string(pwd(), "/Fibo_externC.so")
else
    const libpath = string(pwd(), "/Fibo_externC.dylib")
end

In [ ]:
function cpp_wrapped_fibo(n::Int64)
    r = ccall( (:Fibonacci_I, libpath), # function + library
                    Int64, # return type
                    (Int64,),
                    n)
    return r
end

In [ ]:
cpp_wrapped_fibo(10)